# Wealth Management Dashboard

**Purpose:** Portfolio performance analytics, Time Weighted Return (TWR), asset allocation optimization, risk analytics, and client segmentation for wealth advisors.

**Audience:** Wealth Advisors, Relationship Managers, Head of Private Banking, Investment Advisory Teams, Portfolio Managers

---

## The Client Performance Request Scenario

**Monday, 9:00 AM.** PLATINUM client calls wealth advisor:

**Client:** "I need my portfolio performance report for my tax advisor by tomorrow. Can you send it?"

**Questions:**
1. **What's my YTD return and how does it compare to benchmarks?**
2. **What's my risk-adjusted performance (Sharpe ratio)?**
3. **Is my asset allocation still optimal for my risk profile?**
4. **How much have I paid in commissions?**

**Traditional Response:** "I'll send it by end of day..." (Spend 3 hours manually calculating in Excel. Discover formula error. Send report 2 days late. Client: "My friend got theirs in 30 minutes. I'm moving my €2.4M elsewhere." €18K annual fee lost.)

**With This Notebook:** "Let me pull that up right now..." (Run query in 30 seconds. Export to PDF. Email sent in 5 minutes. Client: "That was incredible! Actually, I have €500K from a property sale - can you invest it?" €3.75K additional annual fee secured.)

---

## Daily Routine: Wealth Advisor Workflow

### 8:00 AM - 9:00 AM: Portfolio Review Preparation
- Run Portfolio Performance Dashboard for day's meetings
- Check client account balances and recent transactions
- Review asset allocation drift (rebalancing opportunities)
- Prepare risk-adjusted return comparisons

### 9:00 AM - 5:00 PM: Client Meetings & Advisory
- Present real-time performance metrics
- Discuss rebalancing recommendations
- Analyze risk-return trade-offs
- Identify cross-sell opportunities

### End of Day: Reporting & Planning
- Generate performance reports for client requests
- Export dashboards for regulatory/internal reviews
- Plan next day's client meetings

---

## 1. Environment Setup

**Business Context:** Connect to wealth management analytics platform for portfolio performance analysis.

**Data Sources:**
- `ACCA_AGG_DT_ACCOUNTS` - Account master data and current balances
- `PAYA_AGG_DT_ACCOUNT_BALANCES` - Real-time account balance tracking
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Transaction history for performance calculation
- `CRMA_AGG_DT_CUSTOMER_360` - Customer profiles and asset allocation
- `EMPA_AGG_DT_PORTFOLIO_BY_ADVISOR` - Portfolio valuations by advisor


In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('PUBLIC')
st.success('Connected to Wealth Management Analytics Platform')


## 3. Time-Weighted Return (TWR) Analysis

**Business Question:** "What's my year-to-date portfolio return and how does it compare to my peers?"

**Why This Matters:**
- **Performance Transparency:** Provide accurate, time-weighted returns that account for cash flows
- **Client Confidence:** Demonstrate value of wealth management services with concrete numbers
- **Competitive Advantage:** Show performance vs. benchmarks and peer groups
- **Regulatory Compliance:** Accurate performance reporting for MIFID II and other regulations

**What We're Measuring:**
- Time-Weighted Return (TWR) by account and customer
- Annualized returns for multi-period comparison
- Return breakdown by asset class (equity, fixed income, commodities)
- Portfolio value growth over measurement period


In [ ]:
# Query: Time-Weighted Return (TWR) by Client
query_twr = '''
WITH top_performers AS (
    SELECT 
        p.CUSTOMER_ID,
        c.FIRST_NAME || ' ' || c.FAMILY_NAME as CLIENT_NAME,
        c.COUNTRY,
        c.ACCOUNT_TIER,
        COUNT(DISTINCT p.ACCOUNT_ID) as ACCOUNTS,
        SUM(COALESCE(p.EQUITY_TOTAL_INVESTED_CHF, 0) + COALESCE(p.FI_TOTAL_INVESTED_CHF, 0) + COALESCE(p.CMD_TOTAL_INVESTED_CHF, 0)) as TOTAL_INVESTED,
        SUM(p.TOTAL_PORTFOLIO_VALUE_CHF) as CURRENT_VALUE,
        SUM(p.TOTAL_RETURN_CHF) as TOTAL_RETURN,
        
        -- Weighted Average TWR
        ROUND(
            SUM(p.TOTAL_PORTFOLIO_TWR_PERCENTAGE * p.TOTAL_PORTFOLIO_VALUE_CHF) / 
            NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
            2
        ) as PORTFOLIO_TWR_PCT,
        
        -- Weighted Average Annualized TWR
        ROUND(
            SUM(p.ANNUALIZED_PORTFOLIO_TWR * p.TOTAL_PORTFOLIO_VALUE_CHF) / 
            NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
            2
        ) as ANNUALIZED_TWR_PCT,
        
        -- Asset Class Returns
        ROUND(AVG(p.EQUITY_RETURN_PERCENTAGE), 2) as AVG_EQUITY_RETURN_PCT,
        ROUND(AVG(p.FI_RETURN_PERCENTAGE), 2) as AVG_FI_RETURN_PCT,
        ROUND(AVG(p.CMD_RETURN_PERCENTAGE), 2) as AVG_COMMODITY_RETURN_PCT,
        
        MAX(p.CALCULATION_TIMESTAMP) as AS_OF_DATE
        
    FROM REP_AGG_001.REPP_AGG_DT_PORTFOLIO_PERFORMANCE p
    JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c 
        ON p.CUSTOMER_ID = c.CUSTOMER_ID
    WHERE p.TOTAL_PORTFOLIO_VALUE_CHF > 0
    GROUP BY p.CUSTOMER_ID, c.FIRST_NAME, c.FAMILY_NAME, c.COUNTRY, c.ACCOUNT_TIER
)
SELECT *
FROM top_performers
ORDER BY PORTFOLIO_TWR_PCT DESC
LIMIT 50
'''

df_twr = session.sql(query_twr).to_pandas()

if len(df_twr) > 0:
    st.subheader('📈 Top 50 Performers by Time-Weighted Return (TWR)')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric('Avg TWR', f"{df_twr['PORTFOLIO_TWR_PCT'].mean():.2f}%")
    with col2:
        st.metric('Avg Annualized', f"{df_twr['ANNUALIZED_TWR_PCT'].mean():.2f}%")
    with col3:
        st.metric('Top Performer', f"{df_twr['PORTFOLIO_TWR_PCT'].max():.2f}%")
    with col4:
        st.metric('Total Clients', len(df_twr))
    
    # Data table
    st.dataframe(df_twr, use_container_width=True, height=400)
    
    # Chart: TWR Distribution
    st.subheader('TWR Distribution by Client')
    st.bar_chart(df_twr.set_index('CLIENT_NAME')['PORTFOLIO_TWR_PCT'].head(20))
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_twr.to_csv(index=False)
    st.download_button(
        label='📥 Download TWR Report (CSV)',
        data=csv_data,
        file_name=f'TWR_Analysis_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No portfolio performance data available. Ensure REPP_AGG_DT_PORTFOLIO_PERFORMANCE has been refreshed.')


## 4. Risk-Adjusted Performance & Sharpe Ratio

**Business Question:** "Am I getting adequate returns for the risk I'm taking?"

**Why This Matters:**
- **Risk Management:** Higher returns don't always mean better performance if they come with excessive risk
- **Client Suitability:** Ensure portfolios match client risk tolerance
- **Performance Quality:** Identify advisors/strategies that deliver superior risk-adjusted returns
- **Regulatory Requirements:** MIFID II requires suitability assessment including risk/return profiles

**What We're Measuring:**
- Portfolio volatility (standard deviation of returns)
- Sharpe Ratio (excess return per unit of risk)
- Risk-free rate benchmarking
- Risk-adjusted performance rankings


In [ ]:
# Query: Risk-Adjusted Performance Analysis
query_risk = '''
SELECT 
    p.CUSTOMER_ID,
    c.FIRST_NAME || ' ' || c.FAMILY_NAME as CLIENT_NAME,
    c.ACCOUNT_TIER,
    c.RISK_CLASSIFICATION,
    
    -- Portfolio Size
    ROUND(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 2) as TOTAL_AUM,
    
    -- Returns
    ROUND(AVG(p.TOTAL_PORTFOLIO_TWR_PERCENTAGE), 2) as AVG_TWR_PCT,
    ROUND(AVG(p.ANNUALIZED_PORTFOLIO_TWR), 2) as AVG_ANNUALIZED_TWR_PCT,
    
    -- Risk Metrics
    ROUND(AVG(p.PORTFOLIO_VOLATILITY), 2) as AVG_VOLATILITY_PCT,
    ROUND(AVG(p.SHARPE_RATIO), 2) as AVG_SHARPE_RATIO,
    ROUND(AVG(p.RISK_FREE_RATE_ANNUAL_PCT), 2) as RISK_FREE_RATE_PCT,
    
    -- Risk Classification
    CASE 
        WHEN AVG(p.SHARPE_RATIO) >= 2.0 THEN 'EXCELLENT'
        WHEN AVG(p.SHARPE_RATIO) >= 1.5 THEN 'VERY_GOOD'
        WHEN AVG(p.SHARPE_RATIO) >= 1.0 THEN 'GOOD'
        WHEN AVG(p.SHARPE_RATIO) >= 0.5 THEN 'FAIR'
        WHEN AVG(p.SHARPE_RATIO) > 0 THEN 'POOR'
        ELSE 'NEGATIVE'
    END as PERFORMANCE_RATING,
    
    COUNT(DISTINCT p.ACCOUNT_ID) as ACCOUNTS

FROM REP_AGG_001.REPP_AGG_DT_PORTFOLIO_PERFORMANCE p
JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c 
    ON p.CUSTOMER_ID = c.CUSTOMER_ID
WHERE p.TOTAL_PORTFOLIO_VALUE_CHF > 0
  AND p.TOTAL_PORTFOLIO_TWR_PERCENTAGE IS NOT NULL
GROUP BY p.CUSTOMER_ID, c.FIRST_NAME, c.FAMILY_NAME, c.ACCOUNT_TIER, c.RISK_CLASSIFICATION
HAVING COUNT(DISTINCT p.ACCOUNT_ID) > 0
ORDER BY AVG_SHARPE_RATIO DESC NULLS LAST
LIMIT 50
'''

df_risk = session.sql(query_risk).to_pandas()

if len(df_risk) > 0:
    st.subheader('🎯 Risk-Adjusted Performance (Sharpe Ratio)')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        avg_sharpe = df_risk['AVG_SHARPE_RATIO'].mean()
        st.metric('Avg Sharpe Ratio', f"{avg_sharpe:.2f}" if not pd.isna(avg_sharpe) else "N/A")
    with col2:
        avg_vol = df_risk['AVG_VOLATILITY_PCT'].mean()
        st.metric('Avg Volatility', f"{avg_vol:.2f}%" if not pd.isna(avg_vol) else "N/A")
    with col3:
        excellent_count = len(df_risk[df_risk['PERFORMANCE_RATING'] == 'EXCELLENT'])
        st.metric('Excellent Performers', excellent_count)
    with col4:
        total_aum = df_risk['TOTAL_AUM'].sum()
        st.metric('Total AUM', f"${total_aum:,.0f}")
    
    # Info banner
    st.info('''
    **Sharpe Ratio Interpretation:**
    - **>2.0:** Excellent - Outstanding risk-adjusted returns
    - **1.5-2.0:** Very Good - Strong performance for the risk taken
    - **1.0-1.5:** Good - Acceptable risk-adjusted returns
    - **0.5-1.0:** Fair - Returns barely compensate for risk
    - **<0.5:** Poor/Negative - Inadequate compensation for risk
    ''')
    
    # Data table
    st.dataframe(df_risk, use_container_width=True, height=400)
    
    # Chart: Sharpe Ratio by Account Tier
    if 'ACCOUNT_TIER' in df_risk.columns:
        st.subheader('Sharpe Ratio by Account Tier')
        tier_sharpe = df_risk.groupby('ACCOUNT_TIER')['AVG_SHARPE_RATIO'].mean().sort_values(ascending=False)
        st.bar_chart(tier_sharpe)
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_risk.to_csv(index=False)
    st.download_button(
        label='📥 Download Risk Analysis (CSV)',
        data=csv_data,
        file_name=f'Risk_Adjusted_Performance_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No risk metrics available. Ensure portfolio performance data has been calculated.')


## 5. Asset Allocation Analysis

**Business Question:** "Is my portfolio properly diversified across asset classes?"

**Why This Matters:**
- **Diversification:** Reduce portfolio risk through proper asset allocation
- **Rebalancing Opportunities:** Identify portfolios that have drifted from target allocations
- **Suitability:** Ensure asset mix matches client risk profile and investment goals
- **Revenue Generation:** Rebalancing generates trading commissions

**What We're Measuring:**
- Current allocation across cash, equity, fixed income, and commodities
- Concentration risk (over-allocation to single asset class)
- Asset allocation by wealth segment
- Rebalancing recommendations


In [ ]:
# Query: Asset Allocation Analysis
query_allocation = '''
SELECT 
    p.CUSTOMER_ID,
    c.FIRST_NAME || ' ' || c.FAMILY_NAME as CLIENT_NAME,
    c.ACCOUNT_TIER,
    c.RISK_CLASSIFICATION,
    
    -- Total Portfolio Value
    ROUND(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 2) as TOTAL_PORTFOLIO_VALUE,
    
    -- Asset Class Values
    ROUND(SUM(p.CURRENT_CASH_VALUE_CHF), 2) as CASH_VALUE,
    ROUND(SUM(p.CURRENT_EQUITY_VALUE_CHF), 2) as EQUITY_VALUE,
    ROUND(SUM(p.CURRENT_FI_VALUE_CHF), 2) as FI_VALUE,
    ROUND(SUM(p.CURRENT_CMD_VALUE_CHF), 2) as COMMODITY_VALUE,
    
    -- Asset Allocation Percentages
    ROUND(
        SUM(p.CURRENT_CASH_VALUE_CHF) * 100.0 / 
        NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
        2
    ) as CASH_ALLOCATION_PCT,
    
    ROUND(
        SUM(p.CURRENT_EQUITY_VALUE_CHF) * 100.0 / 
        NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
        2
    ) as EQUITY_ALLOCATION_PCT,
    
    ROUND(
        SUM(p.CURRENT_FI_VALUE_CHF) * 100.0 / 
        NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
        2
    ) as FI_ALLOCATION_PCT,
    
    ROUND(
        SUM(p.CURRENT_CMD_VALUE_CHF) * 100.0 / 
        NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
        2
    ) as COMMODITY_ALLOCATION_PCT,
    
    -- Concentration Risk Flag
    CASE 
        WHEN GREATEST(
            SUM(p.CURRENT_CASH_VALUE_CHF),
            SUM(p.CURRENT_EQUITY_VALUE_CHF),
            SUM(p.CURRENT_FI_VALUE_CHF),
            SUM(p.CURRENT_CMD_VALUE_CHF)
        ) * 100.0 / NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0) > 80 
        THEN 'HIGH_CONCENTRATION'
        WHEN GREATEST(
            SUM(p.CURRENT_CASH_VALUE_CHF),
            SUM(p.CURRENT_EQUITY_VALUE_CHF),
            SUM(p.CURRENT_FI_VALUE_CHF),
            SUM(p.CURRENT_CMD_VALUE_CHF)
        ) * 100.0 / NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0) > 60 
        THEN 'MEDIUM_CONCENTRATION'
        ELSE 'WELL_DIVERSIFIED'
    END as CONCENTRATION_RISK

FROM REP_AGG_001.REPP_AGG_DT_PORTFOLIO_PERFORMANCE p
JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c 
    ON p.CUSTOMER_ID = c.CUSTOMER_ID
WHERE p.TOTAL_PORTFOLIO_VALUE_CHF > 0
GROUP BY p.CUSTOMER_ID, c.FIRST_NAME, c.FAMILY_NAME, c.ACCOUNT_TIER, c.RISK_CLASSIFICATION
ORDER BY TOTAL_PORTFOLIO_VALUE DESC
LIMIT 100
'''

df_allocation = session.sql(query_allocation).to_pandas()

if len(df_allocation) > 0:
    st.subheader('🎨 Asset Allocation Analysis')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        avg_equity = df_allocation['EQUITY_ALLOCATION_PCT'].mean()
        st.metric('Avg Equity %', f"{avg_equity:.1f}%")
    with col2:
        avg_fi = df_allocation['FI_ALLOCATION_PCT'].mean()
        st.metric('Avg Fixed Income %', f"{avg_fi:.1f}%")
    with col3:
        avg_cash = df_allocation['CASH_ALLOCATION_PCT'].mean()
        st.metric('Avg Cash %', f"{avg_cash:.1f}%")
    with col4:
        high_conc = len(df_allocation[df_allocation['CONCENTRATION_RISK'] == 'HIGH_CONCENTRATION'])
        st.metric('High Concentration', high_conc)
    
    # Aggregate Allocation Chart
    st.subheader('Overall Asset Allocation Mix')
    total_cash = df_allocation['CASH_VALUE'].sum()
    total_equity = df_allocation['EQUITY_VALUE'].sum()
    total_fi = df_allocation['FI_VALUE'].sum()
    total_cmd = df_allocation['COMMODITY_VALUE'].sum()
    
    allocation_data = pd.DataFrame({
        'Asset Class': ['Cash', 'Equity', 'Fixed Income', 'Commodities'],
        'Value': [total_cash, total_equity, total_fi, total_cmd]
    })
    st.bar_chart(allocation_data.set_index('Asset Class'))
    
    # Data table
    st.dataframe(df_allocation, use_container_width=True, height=400)
    
    # Concentration Risk Analysis
    st.subheader('Concentration Risk Distribution')
    conc_counts = df_allocation['CONCENTRATION_RISK'].value_counts()
    st.bar_chart(conc_counts)
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_allocation.to_csv(index=False)
    st.download_button(
        label='📥 Download Asset Allocation (CSV)',
        data=csv_data,
        file_name=f'Asset_Allocation_Analysis_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No asset allocation data available.')


## 6. Fee & Commission Analysis

**Business Question:** "How much am I paying in fees and commissions, and is it justified by performance?"

**Why This Matters:**
- **Cost Transparency:** Clients increasingly demand visibility into total cost of ownership
- **Competitive Pricing:** Ensure fees are competitive vs. market rates
- **Revenue Analytics:** Track fee income by client, product, and advisor
- **Regulatory Compliance:** MIFID II requires detailed cost disclosure

**What We're Measuring:**
- Total commissions paid by client and asset class
- Commission-to-AUM ratios
- Trading activity and turnover
- Cost efficiency analysis


In [ ]:
# Query: Fee and Commission Analysis
query_fees = '''
SELECT 
    p.CUSTOMER_ID,
    c.FIRST_NAME || ' ' || c.FAMILY_NAME as CLIENT_NAME,
    c.ACCOUNT_TIER,
    
    -- Portfolio Metrics
    ROUND(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 2) as TOTAL_AUM,
    ROUND(SUM(p.TOTAL_RETURN_CHF), 2) as TOTAL_RETURN,
    ROUND(AVG(p.TOTAL_PORTFOLIO_TWR_PERCENTAGE), 2) as AVG_TWR_PCT,
    
    -- Commission Breakdown by Asset Class
    ROUND(SUM(p.EQUITY_COMMISSION_CHF), 2) as EQUITY_COMMISSIONS,
    ROUND(SUM(p.FI_COMMISSION_CHF), 2) as FI_COMMISSIONS,
    ROUND(SUM(p.CMD_COMMISSION_CHF), 2) as COMMODITY_COMMISSIONS,
    ROUND(SUM(COALESCE(p.EQUITY_COMMISSION_CHF, 0) + COALESCE(p.FI_COMMISSION_CHF, 0) + COALESCE(p.CMD_COMMISSION_CHF, 0)), 2) as TOTAL_COMMISSIONS,
    
    -- Cost Ratios
    ROUND(
        SUM(COALESCE(p.EQUITY_COMMISSION_CHF, 0) + COALESCE(p.FI_COMMISSION_CHF, 0) + COALESCE(p.CMD_COMMISSION_CHF, 0)) * 100.0 / 
        NULLIF(SUM(p.TOTAL_PORTFOLIO_VALUE_CHF), 0), 
        4
    ) as COMMISSION_TO_AUM_PCT,
    
    ROUND(
        SUM(COALESCE(p.EQUITY_COMMISSION_CHF, 0) + COALESCE(p.FI_COMMISSION_CHF, 0) + COALESCE(p.CMD_COMMISSION_CHF, 0)) * 100.0 / 
        NULLIF(ABS(SUM(p.TOTAL_RETURN_CHF)), 0.01), 
        2
    ) as COMMISSION_TO_RETURN_PCT,
    
    -- Trading Activity
    SUM(p.EQUITY_TRADES_COUNT) as EQUITY_TRADES,
    SUM(p.FI_TRADES_COUNT) as FI_TRADES,
    SUM(p.CMD_TRADES_COUNT) as COMMODITY_TRADES,
    
    -- Net Performance (After Fees)
    ROUND(SUM(p.TOTAL_RETURN_CHF) - SUM(COALESCE(p.EQUITY_COMMISSION_CHF, 0) + COALESCE(p.FI_COMMISSION_CHF, 0) + COALESCE(p.CMD_COMMISSION_CHF, 0)), 2) as NET_RETURN_AFTER_FEES

FROM REP_AGG_001.REPP_AGG_DT_PORTFOLIO_PERFORMANCE p
JOIN CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c 
    ON p.CUSTOMER_ID = c.CUSTOMER_ID
WHERE p.TOTAL_PORTFOLIO_VALUE_CHF > 0
GROUP BY p.CUSTOMER_ID, c.FIRST_NAME, c.FAMILY_NAME, c.ACCOUNT_TIER
HAVING SUM(COALESCE(p.EQUITY_COMMISSION_CHF, 0) + COALESCE(p.FI_COMMISSION_CHF, 0) + COALESCE(p.CMD_COMMISSION_CHF, 0)) > 0
ORDER BY TOTAL_COMMISSIONS DESC
LIMIT 100
'''

df_fees = session.sql(query_fees).to_pandas()

if len(df_fees) > 0:
    st.subheader('💰 Fee & Commission Analysis')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        total_comm = df_fees['TOTAL_COMMISSIONS'].sum()
        st.metric('Total Commissions', f"${total_comm:,.0f}")
    with col2:
        avg_comm_ratio = df_fees['COMMISSION_TO_AUM_PCT'].mean()
        st.metric('Avg Comm/AUM %', f"{avg_comm_ratio:.3f}%")
    with col3:
        total_trades = (df_fees['EQUITY_TRADES'].sum() + 
                       df_fees['FI_TRADES'].sum() + 
                       df_fees['COMMODITY_TRADES'].sum())
        st.metric('Total Trades', f"{int(total_trades):,}")
    with col4:
        net_return = df_fees['NET_RETURN_AFTER_FEES'].sum()
        st.metric('Net Return (After Fees)', f"${net_return:,.0f}")
    
    # Commission Breakdown Chart
    st.subheader('Commission Breakdown by Asset Class')
    commission_breakdown = pd.DataFrame({
        'Asset Class': ['Equity', 'Fixed Income', 'Commodities'],
        'Commissions': [
            df_fees['EQUITY_COMMISSIONS'].sum(),
            df_fees['FI_COMMISSIONS'].sum(),
            df_fees['COMMODITY_COMMISSIONS'].sum()
        ]
    })
    st.bar_chart(commission_breakdown.set_index('Asset Class'))
    
    # Data table
    st.dataframe(df_fees, use_container_width=True, height=400)
    
    # Cost Efficiency Analysis
    st.subheader('Cost Efficiency: Commission vs. Return')
    st.info('''
    **Commission-to-Return Ratio Interpretation:**
    - **<10%:** Excellent - Low cost relative to returns generated
    - **10-20%:** Good - Acceptable cost structure
    - **20-30%:** Fair - Consider fee negotiation
    - **>30%:** High - Fees consuming significant portion of returns
    ''')
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_fees.to_csv(index=False)
    st.download_button(
        label='📥 Download Fee Analysis (CSV)',
        data=csv_data,
        file_name=f'Fee_Commission_Analysis_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No commission data available.')


GG

In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('PUBLIC')
st.success('Connected to Wealth Management Analytics Platform')

## 2. Portfolio Overview Dashboard

**Business Question:** "What's the performance and composition of my client portfolios?"

**Why This Matters:**
- **Client Satisfaction:** Provide real-time portfolio transparency
- **Advisory Excellence:** Data-driven investment recommendations  
- **Revenue Optimization:** Identify cross-sell and upsell opportunities
- **Risk Management:** Monitor portfolio concentration and volatility

**What We're Measuring:**
- Total assets under management (AUM) by client
- Account balance distribution by account type
- Number of accounts per client (product penetration)
- Client segmentation by portfolio size
- Geographic and advisor distribution of AUM


In [ ]:
# Query: Portfolio Overview Dashboard
query = '''
WITH portfolio_summary AS (
    SELECT 
        c.CUSTOMER_ID,
        c.FIRST_NAME || ' ' || c.FAMILY_NAME as CLIENT_NAME,
        c.COUNTRY,
        c.ONBOARDING_DATE,
        c.TOTAL_ACCOUNTS,
        c.TOTAL_BALANCE as TOTAL_AUM,
        c.TOTAL_TRANSACTIONS,
        c.LAST_TRANSACTION_DATE,
        c.RISK_CLASSIFICATION,
        CASE 
            WHEN c.TOTAL_BALANCE >= 10000000 THEN 'ULTRA_HIGH_NET_WORTH'
            WHEN c.TOTAL_BALANCE >= 1000000 THEN 'HIGH_NET_WORTH'
            WHEN c.TOTAL_BALANCE >= 250000 THEN 'AFFLUENT'
            WHEN c.TOTAL_BALANCE >= 50000 THEN 'MASS_AFFLUENT'
            ELSE 'RETAIL'
        END as WEALTH_SEGMENT
    FROM CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c
    WHERE c.TOTAL_ACCOUNTS > 0
)
SELECT 
    -- Overall AUM Metrics
    COUNT(DISTINCT CUSTOMER_ID) as TOTAL_CLIENTS,
    SUM(TOTAL_AUM) as TOTAL_AUM_USD,
    ROUND(AVG(TOTAL_AUM), 2) as AVG_AUM_PER_CLIENT,
    ROUND(MEDIAN(TOTAL_AUM), 2) as MEDIAN_AUM_PER_CLIENT,
    MAX(TOTAL_AUM) as LARGEST_PORTFOLIO,
    
    -- Product Penetration
    SUM(TOTAL_ACCOUNTS) as TOTAL_ACCOUNTS,
    ROUND(AVG(TOTAL_ACCOUNTS), 2) as AVG_ACCOUNTS_PER_CLIENT,
    
    -- Client Segmentation
    COUNT(CASE WHEN WEALTH_SEGMENT = 'ULTRA_HIGH_NET_WORTH' THEN 1 END) as UHNW_CLIENTS,
    COUNT(CASE WHEN WEALTH_SEGMENT = 'HIGH_NET_WORTH' THEN 1 END) as HNW_CLIENTS,
    COUNT(CASE WHEN WEALTH_SEGMENT = 'AFFLUENT' THEN 1 END) as AFFLUENT_CLIENTS,
    COUNT(CASE WHEN WEALTH_SEGMENT = 'MASS_AFFLUENT' THEN 1 END) as MASS_AFFLUENT_CLIENTS,
    COUNT(CASE WHEN WEALTH_SEGMENT = 'RETAIL' THEN 1 END) as RETAIL_CLIENTS,
    
    -- Activity Metrics
    SUM(TOTAL_TRANSACTIONS) as TOTAL_TRANSACTIONS,
    COUNT(CASE WHEN LAST_TRANSACTION_DATE >= DATEADD(day, -30, CURRENT_DATE()) THEN 1 END) as ACTIVE_LAST_30_DAYS
    
FROM portfolio_summary
'''

df_portfolio = session.sql(query).to_pandas()
st.subheader('Portfolio Overview Metrics')
st.dataframe(df_portfolio, use_container_width=True)

# Export
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_data = df_portfolio.to_csv(index=False)
st.download_button(
    label='Download Portfolio Report (CSV)',
    data=csv_data,
    file_name=f'Portfolio_Overview_{ts}.csv',
    mime='text/csv'
)

## 3. Next Steps

**Additional Analyses Available:**

This notebook can be extended with:
- Time-weighted return (TWR) calculations by client
- Risk-adjusted performance metrics (Sharpe ratio, Sortino ratio)
- Asset allocation analysis and rebalancing recommendations
- Client-specific portfolio performance reports
- Benchmark comparison (vs S&P 500, bond indices, etc.)
- Fee and commission analysis by client

**For More Information:**
- Review `structure/ACCA_AGG_DT_ACCOUNTS` for account-level analytics
- See `EMPA_AGG_DT_PORTFOLIO_BY_ADVISOR` for advisor performance metrics
- Contact Head of Wealth Management for specific portfolio analysis requests
